# Jacobian

Lets begin with some definitions

$$
J_{IK} = \frac{\partial x}{\partial(rs)} = \frac{\partial N_i}{\partial(\eta_K)} X_iI
$$

being $X_iI$ the nodal coordinates in the $I$ dimension, the current position and X the reference position, with $i$ in sum

In [16]:
import numpy as np

In [17]:
import numpy as np
# Define material properties
E = 200e9  # Young's modulus in Pa
nu = 0.3   # Poisson's ratio
m_dim = 2
m_nodxelem = 4
# Define element properties
num_nodes_element = 4  # Number of nodes per element
element_length = 1.0   # Length of the element

dNdX = np.zeros((m_dim, m_nodxelem)) 
# Define shape functions and their derivatives for 2D quadrilateral element
def shape_functions(xi, eta):
    N = np.array([(1-xi)*(1-eta)/4,
                  (1+xi)*(1-eta)/4,
                  (1+xi)*(1+eta)/4,
                  (1-xi)*(1+eta)/4])
    dNdX[0,:] = np.array([-(1-eta)/4, (1-eta)/4, (1+eta)/4, -(1+eta)/4])
    dNdX[1,:] = np.array([-(1-xi)/4, -(1+xi)/4, (1+xi)/4, (1-xi)/4])
    return N, dNdX
    print(dNdX)
# Define material matrix for plane stress
def material_matrix():
    C = E / (1 - nu**2) * np.array([[1, nu, 0],
                                     [nu, 1, 0],
                                     [0, 0, (1 - nu) / 2]])
    return C

# Gauss quadrature points and weights
gauss_points = np.array([[-0.577350269, -0.577350269],
                         [ 0.577350269, -0.577350269],
                         [ 0.577350269,  0.577350269],
                         [-0.577350269,  0.577350269]])
gauss_weights = np.array([1, 1, 1, 1])

gp_count = len(gauss_points)

In case of elasticity, we can define $ B^T C B$

In [18]:
# Finite element strain rate calculation
def calculate_jacobian(velocities):
    strain_rate = np.zeros((3, 3))
    J = np.zeros((gp_count, 2, 2))
    for gp in range(len(gauss_points)):
        xi, eta = gauss_points[gp]
        weight = gauss_weights[gp]
        N, dN_dX = shape_functions(xi, eta)
        J[gp] = np.dot(dNdX, velocities)
        detJ = np.linalg.det(J[gp])
        invJ = np.linalg.inv(J[gp])
    return J


In [19]:
# Finite element strain rate calculation
def calculate_jacobian(velocities):
    strain_rate = np.zeros((3, 3))
    J = np.zeros((gp_count, 2, 2))
    for gp in range(len(gauss_points)):
        xi, eta = gauss_points[gp]
        weight = gauss_weights[gp]
        N, dN_dX = shape_functions(xi, eta)
        J[gp] = np.dot(dNdX, velocities)
        detJ = np.linalg.det(J[gp])
        invJ = np.linalg.inv(J[gp])
    return J


Now use it 

In [20]:
# Example usage
x    =  np.array([[0., 0.], [0.1, 0.], [0., 0.1], [0.1, 0.1]])
velocities = np.array([[1, 0], [2, 0], [2, 1], [1, 1]])  # Example velocities at nodes
strain_rate = calculate_strain_rate(velocities)
print("Strain rate:")
print(strain_rate)

Jacobian [[5.00000000e-01 0.00000000e+00]
 [5.55111512e-17 5.00000000e-01]]
Jacobian [[5.00000000e-01 0.00000000e+00]
 [2.77555756e-17 5.00000000e-01]]
Jacobian [[5.00000000e-01 0.00000000e+00]
 [2.77555756e-17 5.00000000e-01]]
Jacobian [[5.00000000e-01 0.00000000e+00]
 [5.55111512e-17 5.00000000e-01]]
Strain rate:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


## Velocity Gradient and Strain Rate
Velocity gradient tensor is defined as:
$$ \nabla v = \frac{dv_I}{dx_J}$$
Due shape function gradients are calculated as gauss points, 
We can express this as 
$$ \nabla v_{IJ}  = \frac{dN_k}{dX_J} X_{kI} $$
This means that, for each dimension, 

In [21]:
# Define nodal velocities (dummy data for demonstration)
vel = np.full(m_dim * m_nodxelem, 0.1)
vel[5] = vel[7] = -1.0

def velocity_gradient_tensor(dNdX, vel):
    grad_v = np.zeros((m_dim, m_dim))
    for I in range(m_dim): 
        for J in range(m_dim):
            for k in range(m_nodxelem): 
                grad_v[I, J] += dNdX[I, k] * vel[k * m_dim + I]
    return grad_v

In [24]:

grad_v = velocity_gradient_tensor(dNdX, vel)
print(grad_v)


[[ 0.    0.  ]
 [-0.55 -0.55]]


In [ ]:
def calculate_strain_rate(grad_v);
    str_rate = 0.5 *(grad_v + grad_v.T)

In [ ]:
# Finite element strain rate calculation
def calculate_strain_rate(velocities):
    strain_rate = np.zeros((3, 3))
    J = np.zeros((gp_count, 2, 2))
    for gp in range(len(gauss_points)):
        J = calculate_jacobian(velocities)
        xi, eta = gauss_points[gp]
        weight = gauss_weights[gp]
        N, dNdX = shape_functions(xi, eta)
        J[gp] = np.dot(dNdX, velocities)
        detJ = np.linalg.det(J[gp])
        invJ = np.linalg.inv(J[gp])
        B = np.zeros((3, 8))
        for i in range(num_nodes_element):
            B[0, 2*i] = dNdX[0,i]
            B[1, 2*i+1] = dNdX[1,i]
            B[2, 2*i] = dNdX[1,i]
            B[2, 2*i+1] = dNdX[0,i]
        C = material_matrix()
        #strain_rate += np.dot(B.T, np.dot(invJ.T, np.dot(B, C))) * detJ * weight
        print ("Jacobian", J[gp])
    strain_rate *= 0.5
    return strain_rate